<a href="https://colab.research.google.com/github/xzxstar107/Feature-basedMDP_NV/blob/main/CMDP_test_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy

In [ ]:
# !wget https://packages.gurobi.com/10.0/gurobi10.0.1_linux64.tar.gz
# !tar -xvf gurobi10.0.1_linux64.tar.gz
# !cd gurobi100/linux64/ && python3 setup.py install

In [ ]:
import os
os.environ['GRB_LICENSE_FILE'] = '/content/gurobi.lic'

In [ ]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [ ]:
def generate_synthetic_data(n, d, n_x, seed=2023):
    np.random.seed(seed)

    # Create covariance matrix Sigma
    Sigma = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            Sigma[i, j] = (199 / 200) ** abs(i - j)

    # Generate X using multivariate normal distribution
    X = np.random.multivariate_normal(mean=np.zeros(d), cov=Sigma, size=n)

    # Discretize X
    X = np.round(X)

    # Define beta_0
    beta_0 = np.array([((-1) ** i) * (200 - i) for i in range(d)])

    # Normalize beta_0 to have L2 norm equal to 25
    beta_0 = (beta_0 / np.linalg.norm(beta_0)) * 25

    # Generate D values
    D = np.zeros((n, n_x))
    for i, x in enumerate(X):
        for j in range(n_x):
            positive_d_count = 0
            while positive_d_count < n_x:
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(x, beta_0)) + 2 * np.exp(-16 * np.dot(x, beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, positive_d_count] = d
                    positive_d_count += 1

    # Discretize D ensuring different values per X
    for i in range(n):
        unique_d_values = np.unique(np.round(D[i, :]))
        while len(unique_d_values) < n_x:
            for j in range(n_x):
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(X[i], beta_0)) + 2 * np.exp(-16 * np.dot(X[i], beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, j] = d
            unique_d_values = np.unique(np.round(D[i, :]))

        D[i, :] = np.round(unique_d_values)

    return X, D

# an updated version of the generate_synthetic_data function with a more distinct beta_0
def generate_synthetic_data(n, d, n_x, seed=2023):
    np.random.seed(seed)

    # Create covariance matrix Sigma
    Sigma = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            Sigma[i, j] = (199 / 200) ** abs(i - j)

    # Generate X using multivariate normal distribution
    X = np.random.multivariate_normal(mean=np.zeros(d), cov=Sigma, size=n)

    # Discretize X
    X = np.round(X)

    # Define beta_0
    beta_0 = np.array([((-1) ** i) * (400 - i) for i in range(d)])  # Increased the constant

    # Normalize beta_0 to have L2 norm equal to 25
    beta_0 = (beta_0 / np.linalg.norm(beta_0)) * 25

    # Generate D values
    D = np.zeros((n, n_x))
    for i, x in enumerate(X):
        for j in range(n_x):
            positive_d_count = 0
            while positive_d_count < n_x:
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(x, beta_0)) + 2 * np.exp(-16 * np.dot(x, beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, positive_d_count] = d
                    positive_d_count += 1

    # Discretize D ensuring different values per X
    for i in range(n):
        unique_d_values = np.unique(np.round(D[i, :]))
        while len(unique_d_values) < n_x:
            for j in range(n_x):
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(X[i], beta_0)) + 2 * np.exp(-16 * np.dot(X[i], beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, j] = d
            unique_d_values = np.unique(np.round(D[i, :]))

        D[i, :] = np.round(unique_d_values)

    return X, D


# Example usage
n = 10
d = 5
n_x = 3
X, D = generate_synthetic_data(n, d, n_x)
print("X:\n", X)
print("D:\n", D, D.shape)
print("mean of D:\n", D.mean(axis=1))


In [ ]:
# Example usage
n = 10
d = 5
n_x = 5
X, D = generate_synthetic_data(n, d, n_x)
# D = D**2

In [ ]:
X

In [ ]:
D

In [ ]:
# Cost function q
def q(y, d, h, b):
    return h * max(0, -y) + b * max(0, y)

def mle(X, D, function_class):
    n = len(X)
    P_hat = np.zeros((n, len(D[0])))

    for i, x in enumerate(X):
        log_likelihood = np.zeros(len(function_class))
        for j, p in enumerate(function_class):
            log_likelihood[j] = np.sum(np.log(p(x, D[i, :])))
        best_function_idx = np.argmax(log_likelihood)
        P_hat[i, :] = function_class[best_function_idx](x, D[i, :])

    return P_hat

def value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat):
    n_y = len(Y)
    n_x = len(X)
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions and policies at each iteration
    value_history = []
    policy_history = []

    for iteration in range(max_iterations):
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history

In [ ]:
# include the time stamp
import time
import numpy as np
import matplotlib.pyplot as plt

def value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat):
    n_y = len(Y)
    n_x = len(X)
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions, policies and cumulative time at each iteration
    value_history = []
    policy_history = []
    cumulative_time_history = []

    cumulative_time = 0.0
    for iteration in range(max_iterations):
        start_time = time.time()
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        cumulative_time += time.time() - start_time
        cumulative_time_history.append(cumulative_time)

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history, cumulative_time_history

In [ ]:
def generate_synthetic_data(n, d, n_x, seed=2023):
    np.random.seed(seed)

    # Create covariance matrix Sigma
    Sigma = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            Sigma[i, j] = (199 / 200) ** abs(i - j)

    # Generate X using multivariate normal distribution
    X = np.random.multivariate_normal(mean=np.zeros(d), cov=Sigma, size=n)

    # Discretize X
    X = np.round(X)

    # Define beta_0
    beta_0 = np.array([((-1) ** i) * (200 - i) for i in range(d)])

    # Normalize beta_0 to have L2 norm equal to 25
    beta_0 = (beta_0 / np.linalg.norm(beta_0)) * 25

    # Generate D values
    D = np.zeros((n, n_x))
    for i, x in enumerate(X):
        for j in range(n_x):
            positive_d_count = 0
            while positive_d_count < n_x:
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(x, beta_0)) + 2 * np.exp(-16 * np.dot(x, beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, positive_d_count] = d
                    positive_d_count += 1

    # Discretize D ensuring different values per X
    for i in range(n):
        unique_d_values = np.unique(np.round(D[i, :]))
        while len(unique_d_values) < n_x:
            for j in range(n_x):
                epsilon = np.random.normal(0, 1)
                d = 1.7*(np.sin(2 * np.dot(X[i], beta_0)) + 2 * np.exp(-16 * np.dot(X[i], beta_0) ** 2) + 1) + epsilon

                if d > 0:
                    D[i, j] = d
            unique_d_values = np.unique(np.round(D[i, :]))

        D[i, :] = np.round(unique_d_values)

    return X, D

In [ ]:
# Gaussian kernel function
def gaussian_kernel(x, D, mu, sigma):
    return np.exp(-0.5 * ((D - mu) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))

# Parametrized function class
def parametrized_function(x, D, mu, sigma):
    n_d = len(D)
    probabilities = np.zeros(n_d)

    for i in range(n_d):
        probabilities[i] = gaussian_kernel(x, D[i], mu, sigma)

    probabilities /= np.sum(probabilities)

    return probabilities

# Define the parameter grid for the MLE estimation
mu_grid = np.linspace(10, 20, 3)
sigma_grid = np.linspace(1, 5, 3)

# Create a list of possible distribution functions for the MLE estimation
function_class = []
for mu in mu_grid:
    for sigma in sigma_grid:
        function_class.append(lambda x, D: parametrized_function(x, D, mu, sigma))

In [ ]:
# Create synthetic data for X and D
n = 20
d = 10
n_x = 5
X, D = generate_synthetic_data(n, d, n_x)

# Estimate P(D|X) using the MLE algorithm
P_hat = mle(X, D, function_class)
print("P_hat:\n", P_hat)

# Value iteration parameters
Y = np.arange(-10, 11, 1)
A = np.arange(0, 11, 1)
gamma = 0.1
h = 0.9
b = 1
c = 0
max_iterations = 1000
epsilon = 1e-6

# # Run the value iteration algorithm
# V_y, policy = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
# Run value_iteration function
V_y, policy, value_history, policy_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
print("V_y:\n", V_y)
print("Policy:\n", policy)

In [ ]:
import pandas as pd

results = []

for i, x in enumerate(X):
    for j, y in enumerate(Y):
        results.append([x, y, V_y[j], D.mean(axis=1)[i], policy[j, i]])

results_df = pd.DataFrame(results, columns=["X", "Y", "V", "meanD", "policy"])
results_df.to_csv("value_iteration_results"+str(h)+str(b)+str(c)+"_nx"+str(n)+str(gamma)+".csv", index=False)


In [ ]:
"value_iteration_results"+str(h)+str(b)+str(c)+".csv"

## Plot


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

In [ ]:
def plot_policy(x, i, D, Y, policy, h, b, c, gamma, file_name):
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(Y, policy[:, i])
    ax.set_xlabel('Inventory Level')
    ax.set_ylabel('Order Quantity')
    mean_D = D.mean(axis=1)[i]
    plot_title = f"Policy for X{i} = {x}, Mean D(X) = {mean_D:.2f}"
    ax.set_title(plot_title)
    ax.grid()

    # Add parameter values as footnote
    footnote = f'h={h}, b={b}, c={c}, gamma={gamma}'
    ax.annotate(footnote, xy=(0.2, -0.1), xycoords='axes fraction', fontsize=10, ha='center', va='bottom')

    # Adjust subplot layout to accommodate the footnote
    plt.subplots_adjust(bottom=0.1)

    plt.savefig(file_name)
    plt.close(fig)

In [ ]:
# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                subdir = f"h={h}_b={b}_c={c}_gamma={gamma}"
                os.makedirs(subdir, exist_ok=True)
                for i, x in enumerate(X):
                    # V_y, policy = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    V_y, policy, value_history, policy_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    file_name = f"{subdir}/policy_X{i}={x}_h={h}_b={b}_c={c}_gamma={gamma}.png"
                    plot_policy(x, i, D[:, i:i+1], Y, policy, h, b, c, gamma, file_name)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def compare_policies(X, D, Y, A, policy, h, b, c, gamma):
    fig = plt.figure(figsize=(10, 6))  # Set figure size
    ax = fig.add_axes([0.1, 0.1, 0.6, 0.8])  # Adjust main plot position and size

    for i, x in enumerate(X):
        ax.plot(Y, policy[:, i], label=f"X{i}, Mean D(X) = {D.mean(axis=1)[i]:.2f}")

    ax.set_xlabel('Inventory Level')
    ax.set_ylabel('Order Quantity')
    ax.set_title('Optimal Policy under Different Context')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  # Adjust legend position

    # Add parameter values as footnote
    footnote = f'h={h}, b={b}, c={c}, gamma={gamma}'
    ax.annotate(footnote, xy=(0.2, -0.1), xycoords='axes fraction', fontsize=10, ha='center', va='bottom')

    # Adjust subplot layout to accommodate the footnote
    plt.subplots_adjust(bottom=0.5)

    return fig

In [ ]:
# Unique policies
import os
import numpy as np
import matplotlib.pyplot as plt

def compare_policies(X, D, Y, A, policy, h, b, c, gamma):
    fig = plt.figure(figsize=(10, 6))  # Set figure size
    ax = fig.add_axes([0.1, 0.1, 0.6, 0.8])  # Adjust main plot position and size

    unique_policies = {}
    for i, x in enumerate(X):
        # Check if the policy is unique (not already plotted)
        policy_str = np.array2string(policy[:, i], separator=',')
        if policy_str not in unique_policies:
            unique_policies[policy_str] = True
            ax.plot(Y, policy[:, i], label=f"X{i}, Mean D(X) = {D.mean(axis=1)[i]:.2f}")

    ax.set_xlabel('Inventory Level')
    ax.set_ylabel('Order Quantity')
    ax.set_title('Optimal Policy under Different Context')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)  # Adjust legend position

    # Add parameter values as footnote
    footnote = f'h={h}, b={b}, c={c}, gamma={gamma}'
    ax.annotate(footnote, xy=(0.2, -0.1), xycoords='axes fraction', fontsize=10, ha='center', va='bottom')

    # Adjust subplot layout to accommodate the footnote
    plt.subplots_adjust(bottom=0.5)

    return fig

In [ ]:
# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

# Create output directory if it doesn't exist
output_dir = "policy_comparison_plots2"
os.makedirs(output_dir, exist_ok=True)

for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                # Call your value_iteration function to get the policy for this combination of parameters
                # V_y, policy = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

                # Create and save the plot
                fig = compare_policies(X, D, Y, A, policy, h, b, c, gamma)
                fig.savefig(os.path.join(output_dir, f'policy_comparison_h_{h}_b_{b}_c_{c}_gamma_{gamma}.png'))
                plt.close(fig)

In [ ]:
def plot_value_function_convergence(value_history):
    num_iterations = len(value_history)

    plt.figure()
    value_differences = [np.abs(value_history[i+1] - value_history[i]).max() for i in range(num_iterations - 1)]
    plt.plot(range(1, num_iterations), value_differences)
    plt.xlabel('Iteration')
    plt.ylabel('Max Absolute Change in Value Function')
    plt.title('Value Function Convergence')
    plt.grid()
    plt.show()

def plot_policy_convergence(policy_history):
    num_iterations = len(policy_history)

    plt.figure()
    policy_differences = [np.sum(policy_history[i+1] != policy_history[i]) for i in range(num_iterations - 1)]
    plt.plot(range(1, num_iterations), policy_differences)
    plt.xlabel('Iteration')
    plt.ylabel('Number of Policy Changes')
    plt.title('Policy Convergence')
    plt.grid()
    plt.show()

def plot_combined_convergence(value_history, policy_history):
    num_iterations = len(value_history)

    fig, ax1 = plt.subplots()
    ax2 = ax1.twinx()

    value_differences = [np.abs(value_history[i+1] - value_history[i]).max() for i in range(num_iterations - 1)]
    ax1.plot(range(1, num_iterations), value_differences, 'b-')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Max Absolute Change in Value Function', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.grid()

    policy_differences = [np.sum(policy_history[i+1] != policy_history[i]) for i in range(num_iterations - 1)]
    ax2.plot(range(1, num_iterations), policy_differences, 'r-')
    ax2.set_ylabel('Number of Policy Changes', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    plt.title('Value Function and Policy Convergence')
    plt.show()

In [ ]:
import time
import pandas as pd

# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

# Initialize dataframe to store results
df = pd.DataFrame(columns=['Algorithm', 'h', 'b', 'c', 'gamma', 'Time', 'Converged_Iteration'])

# Iterate over each value iteration algorithm
for value_iteration in [value_iteration, value_iteration0, value_iteration2]:

    # Iterate over each hyperparameter combination
    for h in h_values:
        for b in b_values:
            for c in c_values:
                for gamma in gamma_values:

                    start_time = time.time()

                    # Run value iteration algorithm and get results
                    V_y, policy, value_history, policy_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

                    end_time = time.time()

                    computation_time = end_time - start_time

                    converged_iteration = len(value_history)

                    # Save results to dataframe
                    df = df.append({
                        'Algorithm': value_iteration.__name__,
                        'h': h,
                        'b': b,
                        'c': c,
                        'gamma': gamma,
                        'Time': computation_time,
                        'Converged_Iteration': converged_iteration
                    }, ignore_index=True)

                    # Plot results
                    plot_value_function_convergence(value_history)
                    plt.savefig(f"{value_iteration.__name__}_h{h}_b{b}_c{c}_gamma{gamma}_value_convergence.png")

                    plot_policy_convergence(policy_history)
                    plt.savefig(f"{value_iteration.__name__}_h{h}_b{b}_c{c}_gamma{gamma}_policy_convergence.png")

                    plot_combined_convergence(value_history, policy_history)
                    plt.savefig(f"{value_iteration.__name__}_h{h}_b{b}_c{c}_gamma{gamma}_combined_convergence.png")

# Save dataframe to csv
df.to_csv("value_iteration_results.csv", index=False)


In [ ]:
# Iterate over all parameter combinations of h, b, c, gamma, and X
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                subdir = f"h={h}_b={b}_c={c}_gamma={gamma}"
                os.makedirs(subdir, exist_ok=True)

                results_dir = f"{subdir}/results"
                os.makedirs(results_dir, exist_ok=True)

                fig_dir = f"{subdir}/figures"
                os.makedirs(fig_dir, exist_ok=True)

                for i, x in enumerate(X):
                    # Run the first value iteration function
                    V_y1, policy1, value_history1, policy_history1 = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    # Run the second value iteration function
                    V_y2, policy2, value_history2, policy_history2 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)
                    # Run the third value iteration function
                    V_y0, policy0, value_history0, policy_history0 = value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon)

                    np.save(f"{results_dir}/V_y1_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y1)
                    np.save(f"{results_dir}/policy1_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy1)
                    np.save(f"{results_dir}/V_y2_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y2)
                    np.save(f"{results_dir}/policy2_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy2)
                    np.save(f"{results_dir}/V_y0_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y0)
                    np.save(f"{results_dir}/policy0_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy0)

                    plot_convergence(i, value_history1, value_history2, value_history0, h, b, c, gamma, fig_dir)

In [ ]:
# Define the function for plotting value function convergence for each algorithm
def plot_value_function_convergence(value_history, function_name):
    num_iterations = len(value_history)

    plt.figure()
    value_differences = [np.abs(value_history[i+1] - value_history[i]).max() for i in range(num_iterations - 1)]
    plt.plot(range(1, num_iterations), value_differences, label=function_name)
    plt.xlabel('Iteration')
    plt.ylabel('Max Absolute Change in Value Function')
    plt.title('Value Function Convergence')
    plt.grid()
    plt.legend()
    plt.show()

# Initialize dataframe to store results
df = pd.DataFrame(columns=['Algorithm', 'h', 'b', 'c', 'gamma', 'Time', 'Converged_Iteration'])

# Initialize a list to store the value histories
value_histories = []
function_names = []

# Iterate over each value iteration algorithm
for value_iteration in [value_iteration, value_iteration0, value_iteration2]:

    # Iterate over each hyperparameter combination
    for h in h_values:
        for b in b_values:
            for c in c_values:
                for gamma in gamma_values:

                    # Run value iteration algorithm and get results
                    V_y, policy, value_history, policy_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

                    computation_time = time_history[-1]

                    converged_iteration = len(value_history)

                    # Save results to dataframe
                    df = df.append({
                        'Algorithm': value_iteration.__name__,
                        'h': h,
                        'b': b,
                        'c': c,
                        'gamma': gamma,
                        'Time': computation_time,
                        'Converged_Iteration': converged_iteration
                    }, ignore_index=True)

                    # Save value history and function name for later plotting
                    value_histories.append(value_history)
                    function_names.append(value_iteration.__name__)

# Now plot the convergence of each function
for i in range(len(value_histories)):
    plot_value_function_convergence(value_histories[i], function_names[i])

# Save dataframe to csv
df.to_csv("value_iteration_results.csv", index=False)


In [ ]:
import time
import matplotlib.pyplot as plt

# Create a list of the functions for easy iteration
value_iterations = [value_iteration, value_iteration0, value_iteration2]

# Initialize a dictionary to store the value histories and time histories for each algorithm
results = {}

for value_iteration in value_iterations:
    # Initialize the value history and time history for this algorithm
    value_history = []
    time_history = []

    # Assume all parameters are predefined and same for all algorithms
    V_y, policy, vh, ph = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

    # Run the value iteration algorithm
    for iteration in range(max_iterations):
        # Record the start time of the iteration
        start_time = time.time()

        # Run the value iteration
        V_y, policy, vh_i, ph_i = value_iteration(X, D, Y, A, gamma, h, b, c, iteration, epsilon, P_hat)

        # Record the end time of the iteration
        end_time = time.time()

        # Store the value history and the computation time of this iteration
        value_history.append(vh_i)
        time_history.append(end_time - start_time)

    # Store the value history and time history for this algorithm
    results[value_iteration.__name__] = (value_history, time_history)

# Plot the value function convergence versus computation time for each algorithm
for algorithm, (value_history, time_history) in results.items():
    plt.figure()

    # Plot value function convergence versus computation time
    for i in range(1, len(value_history)):
        value_difference = np.abs(value_history[i] - value_history[i-1]).max()
        plt.plot(time_history[i], value_difference, label=f'Iteration {i}')

    plt.xlabel('Computation Time (s)')
    plt.ylabel('Max Absolute Change in Value Function')
    plt.title(f'Value Function Convergence for {algorithm}')
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt

# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

In [ ]:
def plot_convergence(i, time_history0, time_history1, time_history2, value_history0, value_history1, value_history2, h, b, c, gamma, fig_dir):
    plt.figure(figsize=(10, 7))

    if len(time_history0) == len(value_history0):
        plt.plot(time_history0, [V[-1] for V in value_history0], label="VI 0")

    if len(time_history1) == len(value_history1):
        plt.plot(time_history1, [V[-1] for V in value_history1], label="VI 1")

    if len(time_history2) == len(value_history2):
        plt.plot(time_history2, [V[-1] for V in value_history2], label="VI 2")

    plt.xlabel("Elapsed Time (s)")
    plt.ylabel("Last Value Function")
    plt.title(f"Value Function Convergence over Time for X{i}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/value_convergence_over_time_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()

In [ ]:


# Initialize a dictionary to store the results for each algorithm and hyperparameters combination
results = {}

# Run each value iteration function separately
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                for func in ['value_iteration', 'value_iteration0', 'value_iteration2']:
                    # Initialize the value history and time history for this algorithm
                    value_history = []
                    time_history = []

                    # Run the value iteration algorithm
                    for iteration in range(max_iterations):
                        # Record the start time of the iteration
                        start_time = time.time()

                        # Run the value iteration
                        if func == 'value_iteration':
                            V_y, policy, vh_i, ph_i = value_iteration(X, D, Y, A, gamma, h, b, c, iteration, epsilon, P_hat)
                        elif func == 'value_iteration0':
                            V_y, policy, vh_i, ph_i = value_iteration0(D, Y, A, gamma, h, b, c, iteration, epsilon)
                        elif func == 'value_iteration2':
                            V_y, policy, vh_i, ph_i = value_iteration2(X, D, Y, A, gamma, h, b, c, iteration, epsilon)

                        # Record the end time of the iteration
                        end_time = time.time()

                        # Store the value history and the computation time of this iteration
                        value_history.append(vh_i)
                        time_history.append(end_time - start_time)

                    # Store the value history and time history for this algorithm
                    results[(func, h, b, c, gamma)] = (value_history, time_history)

# Convert results to a pandas DataFrame and save as csv
results_df = pd.DataFrame(results)
results_df.to_csv('value_iteration_results.csv')

# Create a figure for the plots
plt.figure(figsize=(10,8))

# Plot the value function convergence versus computation time for each algorithm and hyperparameters combination
for (algorithm, h, b, c, gamma), (value_history, time_history) in results.items():

    # Plot value function convergence versus computation time
    value_differences = [np.abs(value_history[i] - value_history[i-1]).max() for i in range(1, len(value_history))]
    plt.plot(time_history[1:], value_differences, label=f'{algorithm}, h={h}, b={b}, c={c}, gamma={gamma}')

plt.xlabel('Computation Time (s)')
plt.ylabel('Max Absolute Change in Value Function')
plt.title('Value Function Convergence Comparison')
plt.legend()
plt.grid()
plt.savefig('value_function_convergence_comparison.png')
plt.close()

In [ ]:
def plot_convergence(i, cumulative_time_history0, cumulative_time_history1, cumulative_time_history2, value_history0, value_history1, value_history2, h, b, c, gamma, fig_dir):
    plt.figure(figsize=(10, 7))

    if len(cumulative_time_history0) == len(value_history0):
        plt.plot(cumulative_time_history0, [22-V[-1] for V in value_history0], label="Our")

    if len(cumulative_time_history1) == len(value_history1):
        plt.plot(cumulative_time_history1, [22-V[-1] for V in value_history1], label="gMDP")

    if len(cumulative_time_history2) == len(value_history2):
        plt.plot(cumulative_time_history2, [22-V[-1] for V in value_history2], label="cMDP")

    plt.xlabel("Cumulative Elapsed Time (s)")
    plt.ylabel("Value Function")
    plt.title(f"Value Function Convergence over Time for X{i}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/value_convergence_over_time_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()

In [ ]:
# Iterate over all parameter combinations of h, b, c, gamma, and X
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                subdir = f"h={h}_b={b}_c={c}_gamma={gamma}"
                os.makedirs(subdir, exist_ok=True)

                results_dir = f"{subdir}/results"
                os.makedirs(results_dir, exist_ok=True)

                fig_dir = f"{subdir}/figures"
                os.makedirs(fig_dir, exist_ok=True)

                for i, x in enumerate(X):
                    # Run the three value iteration functions
                    V_y, policy, value_history, policy_history, cumulative_time_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    V_y0, policy0, value_history0, policy_history0, cumulative_time_history0 = value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon)
                    V_y2, policy2, value_history2, policy_history2, cumulative_time_history2 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)

                    plot_convergence(i, cumulative_time_history0, cumulative_time_history, cumulative_time_history2, value_history0, value_history, value_history2, h, b, c, gamma, fig_dir)


In [ ]:
def plot_value_function_convergence(value_histories, labels):
    plt.figure()

    for value_history, label in zip(value_histories, labels):
        num_iterations = len(value_history)
        value_differences = [np.abs(value_history[i+1] - value_history[i]).max() for i in range(num_iterations - 1)]
        plt.plot(range(1, num_iterations), value_differences, label=label)

    plt.xlabel('Iteration')
    plt.ylabel('Max Absolute Change in Value Function')
    plt.title('Value Function Convergence')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# Initialize dataframe to store results and lists to hold value histories
df = pd.DataFrame(columns=['Algorithm', 'h', 'b', 'c', 'gamma', 'Time', 'Converged_Iteration'])

# Let's say that you have three value iteration functions: value_iteration1, value_iteration2, value_iteration3
value_histories_1 = []
value_histories_2 = []
value_histories_3 = []

# Iterate over each value iteration algorithm
value_iterations = [value_iteration, value_iteration0, value_iteration2]
for idx, value_iteration in enumerate(value_iterations):

    # Iterate over each hyperparameter combination
    for h in h_values:
        for b in b_values:
            for c in c_values:
                for gamma in gamma_values:

                    # Run value iteration algorithm and get results
                    V_y, policy, value_history, policy_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

                    computation_time = time_history[-1]

                    converged_iteration = len(value_history)

                    # Save results to dataframe
                    df = df.append({
                        'Algorithm': value_iteration.__name__,
                        'h': h,
                        'b': b,
                        'c': c,
                        'gamma': gamma,
                        'Time': computation_time,
                        'Converged_Iteration': converged_iteration
                    }, ignore_index=True)

                    # Save value history depending on the value iteration function
                    if idx == 0:
                        value_histories_1.append(value_history)
                    elif idx == 1:
                        value_histories_2.append(value_history)
                    elif idx == 2:
                        value_histories_3.append(value_history)

# Call the plotting function after all computations are complete
plot_value_function_convergence([value_histories_1, value_histories_2, value_histories_3], ['value_iteration1', 'value_iteration2', 'value_iteration3'])

# Save dataframe to csv
df.to_csv("value_iteration_results.csv", index=False)


In [ ]:
import time
import pandas as pd

# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]

# Initialize dataframe to store results
df = pd.DataFrame(columns=['Algorithm', 'h', 'b', 'c', 'gamma', 'Time', 'Converged_Iteration'])

# Store value function differences for all algorithms
value_function_differences = []

# Iterate over each hyperparameter combination
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:

                # Handle value_iteration
                start_time = time.time()
                V_y, policy, value_history, policy_history,_ = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                end_time = time.time()
                computation_time = end_time - start_time
                converged_iteration = len(value_history)
                df = df.append({
                    'Algorithm': value_iteration.__name__,
                    'h': h,
                    'b': b,
                    'c': c,
                    'gamma': gamma,
                    'Time': computation_time,
                    'Converged_Iteration': converged_iteration
                }, ignore_index=True)
                value_function_differences.append({'name': 'value_iteration', 'values': [np.abs(value_history[i+1] - value_history[i]).max() for i in range(len(value_history) - 1)]})

# Save dataframe to csv
df.to_csv("value_iteration_results.csv", index=False)

# Plot value function convergence for all functions
plt.figure()
for diff in value_function_differences:
    plt.plot(range(1, len(diff['values'])+1), diff['values'], label=diff['name'])
plt.xlabel('Iteration')
plt.ylabel('Max Absolute Change in Value Function')
plt.title('Value Function Convergence')
plt.legend()
plt.grid()
plt.show()


In [ ]:
def plot_value_function_convergence(value_histories, labels):
    plt.figure()

    for value_history, label in zip(value_histories, labels):
        num_iterations = len(value_history)
        value_differences = [np.abs(value_history[i+1] - value_history[i]).max() for i in range(num_iterations - 1)]
        plt.plot(range(1, num_iterations), value_differences, label=label)

    plt.xlabel('Iteration')
    plt.ylabel('Max Absolute Change in Value Function')
    plt.title('Value Function Convergence')
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Initialize dataframe to store results and lists to hold value histories
df = pd.DataFrame(columns=['Algorithm', 'h', 'b', 'c', 'gamma', 'Time', 'Converged_Iteration'])

# Let's say that you have three value iteration functions: value_iteration1, value_iteration2, value_iteration3
value_histories_1 = []
value_histories_2 = []
value_histories_3 = []

# Iterate over each value iteration algorithm
value_iterations = [value_iteration, value_iteration0, value_iteration2]
for idx, value_iteration in enumerate(value_iterations):

    # Iterate over each hyperparameter combination
    for h in h_values:
        for b in b_values:
            for c in c_values:
                for gamma in gamma_values:

                    # Run value iteration algorithm and get results
                    V_y, policy, value_history, policy_history,_ = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)

                    computation_time = time_history[-1]

                    converged_iteration = len(value_history)

                    # Save results to dataframe
                    df = df.append({
                        'Algorithm': value_iteration.__name__,
                        'h': h,
                        'b': b,
                        'c': c,
                        'gamma': gamma,
                        'Time': computation_time,
                        'Converged_Iteration': converged_iteration
                    }, ignore_index=True)

                    # Save value history depending on the value iteration function
                    if idx == 0:
                        value_histories_1.append(value_history)
                    elif idx == 1:
                        value_histories_2.append(value_history)
                    elif idx == 2:
                        value_histories_3.append(value_history)

# Call the plotting function after all computations are complete
plot_value_function_convergence([value_histories_1, value_histories_2, value_histories_3], ['value_iteration1', 'value_iteration2', 'value_iteration3'])

# Save dataframe to csv
df.to_csv("value_iteration_results.csv", index=False)


In [ ]:
# Iterate over all parameter combinations of h, b, c, gamma, and X
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                subdir = f"h={h}_b={b}_c={c}_gamma={gamma}"
                os.makedirs(subdir, exist_ok=True)

                results_dir = f"{subdir}/results"
                os.makedirs(results_dir, exist_ok=True)

                fig_dir = f"{subdir}/figures"
                os.makedirs(fig_dir, exist_ok=True)

                for i, x in enumerate(X):
                    # Run the three value iteration functions
                    V_y, policy, value_history, policy_history, cumulative_time_history = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    V_y0, policy0, value_history0, policy_history0, cumulative_time_history0 = value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon)
                    V_y2, policy2, value_history2, policy_history2, cumulative_time_history2 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)

                    # # Save the results
                    # for vi, time_history, value_history in zip(range(3), [time_history0, time_history1, time_history2], [value_history0, value_history1, value_history2]):
                    #     print(len(time_history), len(value_history))  # Add this line
                        # df = pd.DataFrame({'time': time_history, 'value': [np.max(V) for V in value_history]})
                        # df.to_csv(f"{results_dir}/VI{vi}_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.csv", index=False)

                    # for vi, time_history, value_history in zip(range(3), [time_history0, time_history1, time_history2], [value_history0, value_history1, value_history2]):
                    #     df = pd.DataFrame({'time': time_history, 'value': [np.max(V) for V in value_history]})
                    #     df.to_csv(f"{results_dir}/VI{vi}_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.csv", index=False)
                    # value_history0, value_history, value_history2 = 22-value_history0, 22-value_history, 22-value_history2

                    value_histories = [value_history, value_history0, value_history2]
                    labels = ["Our", "gMDP", "cMDP"]
                    plot_value_function_convergence(value_histories, labels)


# Save the results

In [ ]:
pwd

In [ ]:
from google.colab import files
import os

# Set the path of the folder containing the files to download
folder_path = '/content/policy_comparison_plots'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Download each file in the folder
for file_name in file_list:
    files.download(os.path.join(folder_path, file_name))

In [ ]:
from google.colab import files
import shutil

# Set the path of the folder to download
folder_path = '/content'

# Zip the folder and save as a file
shutil.make_archive('policy_plots', 'zip', folder_path)

# Download the zipped folder
files.download('policy_plots.zip')

## vanilla CMDP

In [ ]:
import numpy as np

# Cost function q
def q(y, d, h, b):
    return h * max(0, -y) + b * max(0, y)

# Define the transition probabilities functions
def transition_probabilities(X, D):
    function_class = []
    # TODO: Define the function_class based on your specific transition probabilities
    for mu in mu_grid:
        for sigma in sigma_grid:
            function_class.append(lambda x, D: parametrized_function(x, D, mu, sigma))
    return function_class

# Maximum Likelihood Estimation (MLE)
def mle(X, D):
    function_class = transition_probabilities(X, D)
    n = len(X)
    P_hat = np.zeros((n, len(D[0])))

    for i, x in enumerate(X):
        log_likelihood = np.zeros(len(function_class))
        for j, p in enumerate(function_class):
            log_likelihood[j] = np.sum(np.log(p(x, D[i, :])))
        best_function_idx = np.argmax(log_likelihood)
        P_hat[i, :] = function_class[best_function_idx](x, D[i, :])

    return P_hat

# Value Iteration
def value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    n_x = len(X)
    P_hat = mle(X, D)  # estimate P_hat using Maximum Likelihood Estimation
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions and policies at each iteration
    value_history = []
    policy_history = []

    for iteration in range(max_iterations):
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history


In [ ]:
def value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    n_x = len(X)
    P_hat = mle(X, D)  # estimate P_hat using Maximum Likelihood Estimation
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions and policies at each iteration
    value_history = []
    policy_history = []

    time_history = []
    for i in range(max_iterations):
        start_time = time.time()
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)
        end_time = time.time()
        time_history.append(end_time - start_time)
        # Record value and append to value_history
        value_history.append(V.copy())

    return V_y, policy, value_history, policy_history, time_history


In [ ]:
import time
import numpy as np

def value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    V_y = np.zeros(n_y)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_y, dtype=int)

    # Store the value functions, policies and time spent at each iteration
    value_history = []
    policy_history = []
    time_history = []

    for iteration in range(max_iterations):
        start_time = time.time()  # Start time measurement
        for i, y in enumerate(Y):
            min_val = np.inf
            min_a = None

            for a in A:
                cost = c * a
                y_plus = y + a - D

                # Clip Y_plus values to stay within the Y range
                y_plus = np.clip(y_plus, Y[0], Y[-1])

                # Find the index of Y_plus values in the Y array
                y_plus_idx = np.searchsorted(Y, y_plus)

                # Calculate the expected value of the next state
                next_state_value = V_y[y_plus_idx]
                expected_next_state_value = np.mean(next_state_value)

                value = cost + q(y, np.mean(D), h, b) + gamma * expected_next_state_value

                if value < min_val:
                    min_val = value
                    min_a = a

            V_next_y[i] = min_val
            policy[i] = min_a

        time_spent = time.time() - start_time  # Measure time spent
        time_history.append(time_spent)

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history, time_history


def value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    n_x = len(X)
    P_hat = mle(X, D)  # estimate P_hat using Maximum Likelihood Estimation
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions, policies and time spent at each iteration
    value_history = []
    policy_history = []
    time_history = []

    for iteration in range(max_iterations):
        start_time = time.time()  # Start time measurement
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        time_spent = time.time() - start_time  # Measure time spent
        time_history.append(time_spent)

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history, time_history


In [ ]:
import time
import numpy as np

def value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    V_y = np.zeros(n_y)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_y, dtype=int)

    # Store the value functions, policies and cumulative time at each iteration
    value_history = []
    policy_history = []
    cumulative_time_history = []

    cumulative_time = 0.0
    for iteration in range(max_iterations):
        start_time = time.time()
        for i, y in enumerate(Y):
            min_val = np.inf
            min_a = None

            for a in A:
                cost = c * a
                y_plus = y + a - D

                # Clip Y_plus values to stay within the Y range
                y_plus = np.clip(y_plus, Y[0], Y[-1])

                # Find the index of Y_plus values in the Y array
                y_plus_idx = np.searchsorted(Y, y_plus)

                # Calculate the expected value of the next state
                next_state_value = V_y[y_plus_idx]
                expected_next_state_value = np.mean(next_state_value)

                value = cost + q(y, np.mean(D), h, b) + gamma * expected_next_state_value

                if value < min_val:
                    min_val = value
                    min_a = a

            V_next_y[i] = min_val
            policy[i] = min_a

        cumulative_time += time.time() - start_time
        cumulative_time_history.append(cumulative_time)

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history, cumulative_time_history


def value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    n_x = len(X)
    P_hat = mle(X, D)  # estimate P_hat using Maximum Likelihood Estimation
    V_yx = np.zeros((n_y, n_x))
    V_y = np.zeros(n_y)
    V_next_yx = np.zeros_like(V_yx)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_yx, dtype=int)

    # Store the value functions, policies and cumulative time at each iteration
    value_history = []
    policy_history = []
    cumulative_time_history = []

    cumulative_time = 0.0
    for iteration in range(max_iterations):
        start_time = time.time()
        for i, y in enumerate(Y):
            for j, x in enumerate(X):
                min_val = np.inf
                min_a = None

                for a in A:
                    cost = c * a
                    y_plus = y + a - D[j, :]

                    # Clip Y_plus values to stay within the Y range
                    y_plus = np.clip(y_plus, Y[0], Y[-1])

                    # Find the index of Y_plus values in the Y array
                    y_plus_idx = np.searchsorted(Y, y_plus)

                    # Calculate the expected value of the next state, weighted by P(D|X)
                    next_state_value = V_yx[y_plus_idx, j]
                    weighted_next_state_value = np.sum(P_hat[j, :] * next_state_value)

                    value = cost + q(y, D[j, :].mean(), h, b) + gamma * weighted_next_state_value

                    if value < min_val:
                        min_val = value
                        min_a = a

                V_next_yx[i, j] = min_val
                policy[i, j] = min_a

            V_next_y[i] = np.mean(V_next_yx[i, :])

        cumulative_time += time.time() - start_time
        cumulative_time_history.append(cumulative_time)

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_yx = np.copy(V_next_yx)
        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history, cumulative_time_history


## gMDP

In [ ]:
import numpy as np

# Cost function q
def q(y, d, h, b):
    return h * max(0, -y) + b * max(0, y)

def value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    V_y = np.zeros(n_y)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_y, dtype=int)

    # Store the value functions and policies at each iteration
    value_history = []
    policy_history = []

    for iteration in range(max_iterations):
        for i, y in enumerate(Y):
            min_val = np.inf
            min_a = None

            for a in A:
                cost = c * a
                y_plus = y + a - D

                # Clip Y_plus values to stay within the Y range
                y_plus = np.clip(y_plus, Y[0], Y[-1])

                # Find the index of Y_plus values in the Y array
                y_plus_idx = np.searchsorted(Y, y_plus)

                # Calculate the expected value of the next state
                next_state_value = V_y[y_plus_idx]
                expected_next_state_value = np.mean(next_state_value)

                value = cost + q(y, np.mean(D), h, b) + gamma * expected_next_state_value

                if value < min_val:
                    min_val = value
                    min_a = a

            V_next_y[i] = min_val
            policy[i] = min_a

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_y = np.copy(V_next_y)

    return V_y, policy, value_history, policy_history


In [ ]:
def value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon):
    n_y = len(Y)
    V_y = np.zeros(n_y)
    V_next_y = np.zeros_like(V_y)
    policy = np.zeros_like(V_y, dtype=int)

    # Store the value functions and policies at each iteration
    value_history = []
    policy_history = []

    time_history = []
    for i in range(max_iterations):
        start_time = time.time()
        for i, y in enumerate(Y):
            min_val = np.inf
            min_a = None

            for a in A:
                cost = c * a
                y_plus = y + a - D

                # Clip Y_plus values to stay within the Y range
                y_plus = np.clip(y_plus, Y[0], Y[-1])

                # Find the index of Y_plus values in the Y array
                y_plus_idx = np.searchsorted(Y, y_plus)

                # Calculate the expected value of the next state
                next_state_value = V_y[y_plus_idx]
                expected_next_state_value = np.mean(next_state_value)

                value = cost + q(y, np.mean(D), h, b) + gamma * expected_next_state_value

                if value < min_val:
                    min_val = value
                    min_a = a

            V_next_y[i] = min_val
            policy[i] = min_a

        value_history.append(V_next_y.copy())
        policy_history.append(policy.copy())

        if np.abs(V_next_y - V_y).max() < epsilon:
            break

        V_y = np.copy(V_next_y)

        end_time = time.time()
        time_history.append(end_time - start_time)

    return V_y, policy, value_history, policy_history, time_history


## Compare VI

In [ ]:
# Define your parameter ranges
h_values = [1, 2]
b_values = [1, 2]
c_values = [0, 1]
gamma_values = [0.1, 0.9]


In [ ]:
import matplotlib.pyplot as plt
import time

# Assuming the same parameters for all three functions
# X = ...
# D = ...
# Y = ...
# A = ...
# gamma = ...
# h = ...
# b = ...
# c = ...
# max_iterations = ...
# epsilon = ...
# P_hat = ...

# Run the first value iteration function
start = time.time()
V_y1, policy1, value_history1, policy_history1 = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
end = time.time()
print(f"Time for first value iteration: {end - start} seconds")

# Run the second value iteration function
start = time.time()
V_y2, policy2, value_history2, policy_history2 = value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon)
end = time.time()
print(f"Time for second value iteration: {end - start} seconds")

# Run the third value iteration function
start = time.time()
V_y3, policy3, value_history3, policy_history3 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)
end = time.time()
print(f"Time for third value iteration: {end - start} seconds")

# Plot optimal value function versus state y for all three functions
plt.figure()
plt.plot(Y, V_y1, label='value_iteration')
plt.plot(Y, V_y2, label='value_iteration0')
plt.plot(Y, V_y3, label='value_iteration2')
plt.xlabel('State y')
plt.ylabel('Optimal Value Function')
plt.legend()
plt.show()

# Plot value history versus iteration number for all three functions
plt.figure()
plt.plot([np.max(v) for v in value_history1], label='value_iteration')
plt.plot([np.max(v) for v in value_history2], label='value_iteration0')
plt.plot([np.max(v) for v in value_history3], label='value_iteration2')
plt.xlabel('Iteration number')
plt.ylabel('Value History')
plt.legend()
plt.show()


In [ ]:
import os

# Define directory to save results
fig_dir = '/content'
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

# Iterate over all parameter combinations of h, b, c, gamma, and X
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                # Run both value iterations
                V_y1, policy1, value_history1, policy_history1 = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                V_y2, policy2, value_history2, policy_history2 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)

                # Compare outcomes
                difference_in_values = np.abs(V_y1 - V_y2)
                difference_in_policies = np.abs(policy1 - policy2)

                max_diff_values = np.max(difference_in_values)
                max_diff_policies = np.max(difference_in_policies)

                print("Max difference in values: ", max_diff_values)
                print("Max difference in policies: ", max_diff_policies)

                # Write results to file
                with open(os.path.join(fig_dir, 'results.txt'), 'a') as f:
                    f.write(f"For h={h}, b={b}, c={c}, gamma={gamma}:\n")
                    f.write(f"Max difference in values: {max_diff_values}\n")
                    f.write(f"Max difference in policies: {max_diff_policies}\n\n")


## compare q-learning and VI

In [ ]:
import matplotlib.pyplot as plt

# Plotting Value Histories
plt.figure(figsize=(10, 5))

# For simplicity, let's just compare the average value over states at each iteration
avg_value_history_vi1 = [np.mean(values) for values in value_history_vi1]
avg_value_history_vi2 = [np.mean(values) for values in value_history_vi2]
# avg_value_history_ql = [np.mean(values) for values in value_history_ql]

plt.plot(avg_value_history_vi1, label='Value Iteration 1')
plt.plot(avg_value_history_vi2, label='Value Iteration 2')
# plt.plot(avg_value_history_ql, label='Q-Learning')
plt.xlabel('Iteration')
plt.ylabel('Average Value')
plt.title('Average Value Over Iterations')
plt.legend()
plt.grid()
plt.show()

# Plotting Policy Histories
plt.figure(figsize=(10, 5))

# Here we plot the number of changes in the policy at each iteration
policy_changes_vi1 = [np.sum(policy_history_vi1[i] != policy_history_vi1[i-1]) if i > 0 else 0 for i in range(len(policy_history_vi1))]
policy_changes_vi2 = [np.sum(policy_history_vi2[i] != policy_history_vi2[i-1]) if i > 0 else 0 for i in range(len(policy_history_vi2))]

plt.plot(policy_changes_vi1, label='Value Iteration 1')
plt.plot(policy_changes_vi2, label='Value Iteration 2')
# plt.plot(policy_changes_ql, label='Q-Learning')
plt.xlabel('Iteration')
plt.ylabel('Number of Policy Changes')
plt.title('Policy Changes Over Iterations')
plt.legend()
plt.grid()
plt.show()


In [ ]:
def plot_convergence(x, i, value_history1, policy_history1, value_history2, policy_history2, h, b, c, gamma, fig_dir):
    # Value function convergence plot
    plt.figure()
    for j, (V_y_iter1, V_y_iter2) in enumerate(zip(value_history1, value_history2)):
        plt.plot(Y, V_y_iter1[:, i], label=f"Iteration {j} (VI 1)")
        plt.plot(Y, V_y_iter2[:, i], label=f"Iteration {j} (VI 2)", linestyle='--')

    plt.xlabel("Inventory Level")
    plt.ylabel("Value Function")
    plt.title(f"Value Function Convergence for X{i} = {x}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/value_convergence_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()

    # Policy convergence plot
    plt.figure()
    for j, (policy_iter1, policy_iter2) in enumerate(zip(policy_history1, policy_history2)):
        plt.plot(Y, policy_iter1[:, i], label=f"Iteration {j} (VI 1)")
        plt.plot(Y, policy_iter2[:, i], label=f"Iteration {j} (VI 2)", linestyle='--')

    plt.xlabel("Inventory Level")
    plt.ylabel("Order Quantity")
    plt.title(f"Policy Convergence for X{i} = {x}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/policy_convergence_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()

In [ ]:
# Define a function to plot the value and policy histories
def plot_convergence2(i, value_history0, value_history1, value_history2, h, b, c, gamma, fig_dir):
    # Value function convergence plot
    plt.figure(figsize=(10, 7))
    max_values0 = [np.max(V) for V in value_history0]
    max_values1 = [np.max(V) for V in value_history1]
    max_values2 = [np.max(V) for V in value_history2]
    plt.plot(range(len(value_history0)), max_values0, label=f"VI 0")
    plt.plot(range(len(value_history1)), max_values1, label=f"VI 1")
    plt.plot(range(len(value_history2)), max_values2, label=f"VI 2", linestyle='--')

    plt.xlabel("Iteration Number")
    plt.ylabel("Maximum Value Function")
    plt.title(f"Value Function Convergence for X{i}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/value_convergence_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Assuming that you already defined these values somewhere in your code
# X, D, Y, A, gamma_values, h_values, b_values, c_values, max_iterations, epsilon, P_hat

# Define a function to plot the value and policy histories
def plot_convergence(i, value_history1, value_history2, value_history0, h, b, c, gamma, fig_dir):
    # Value function convergence plot
    plt.figure(figsize=(10, 7))
    max_values1 = [np.max(V) for V in value_history1]
    max_values2 = [np.max(V) for V in value_history2]
    max_values0 = [np.max(V) for V in value_history0]
    plt.plot(range(len(value_history1)), max_values1, label=f"VI 1")
    plt.plot(range(len(value_history2)), max_values2, label=f"VI 2", linestyle='--')
    plt.plot(range(len(value_history0)), max_values0, label=f"VI 0", linestyle='-.')

    plt.xlabel("Iteration Number")
    plt.ylabel("Maximum Value Function")
    plt.title(f"Value Function Convergence for X{i}, h={h}, b={b}, c={c}, gamma={gamma}")
    plt.legend()
    plt.savefig(f"{fig_dir}/value_convergence_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.png")
    plt.close()


# Iterate over all parameter combinations of h, b, c, gamma, and X
for h in h_values:
    for b in b_values:
        for c in c_values:
            for gamma in gamma_values:
                subdir = f"h={h}_b={b}_c={c}_gamma={gamma}"
                os.makedirs(subdir, exist_ok=True)

                results_dir = f"{subdir}/results"
                os.makedirs(results_dir, exist_ok=True)

                fig_dir = f"{subdir}/figures"
                os.makedirs(fig_dir, exist_ok=True)

                for i, x in enumerate(X):
                    # Run the first value iteration function
                    V_y1, policy1, value_history1, policy_history1 = value_iteration(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon, P_hat)
                    # Run the second value iteration function
                    V_y2, policy2, value_history2, policy_history2 = value_iteration2(X, D, Y, A, gamma, h, b, c, max_iterations, epsilon)
                    # Run the third value iteration function
                    V_y0, policy0, value_history0, policy_history0 = value_iteration0(D, Y, A, gamma, h, b, c, max_iterations, epsilon)

                    np.save(f"{results_dir}/V_y1_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y1)
                    np.save(f"{results_dir}/policy1_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy1)
                    np.save(f"{results_dir}/V_y2_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y2)
                    np.save(f"{results_dir}/policy2_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy2)
                    np.save(f"{results_dir}/V_y0_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", V_y0)
                    np.save(f"{results_dir}/policy0_X{i}_h={h}_b={b}_c={c}_gamma={gamma}.npy", policy0)

                    plot_convergence(i, value_history1, value_history2, value_history0, h, b, c, gamma, fig_dir)
